In [2]:
# まずまとめて html に変換する。
# Note jupyter のバージョンが新しくないと動かない。 anaconda3-2022.05なら大丈夫


In [2]:
import glob
import subprocess
import os
tools_dir = os.getcwd()


In [3]:
cmd = ['jupyter',  'nbconvert', '--to', 'html', '--output-dir', '../html',
'--Exporter.preprocessors',  'regexremove.MyRegexRemovePreprocessor',
'--MyRegexRemovePreprocessor.input=^# hide$',
'--MyRegexRemovePreprocessor.input=^# hideinput',
'--MyRegexRemovePreprocessor.output=^# hideoutput',
'--MyRegexRemovePreprocessor.all=^# hideall']

env = os.environ
env["PYTHONPATH"] = tools_dir
def ipynb_to_html(file):
    return subprocess.run(cmd + [file], 
                        env=env, 
                        capture_output=True)


In [10]:
notes = glob.glob('../parts/*.ipynb')
for note in notes:
    proc = ipynb_to_html(note)

In [6]:
notes = glob.glob('../parts/[0]*.ipynb')
for note in notes:
    proc = ipynb_to_html(note)

上のコードでHTMLを作成。
下のコードでHTMLからPDFを作成。

In [12]:
# https://degitalization.hatenablog.jp/entry/2021/03/13/102805 から借用

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
 
#driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))


ModuleNotFoundError: No module named 'webdriver_manager'

In [13]:
def PrintSetUp(output_dir):
    #印刷としてPDF保存する設定
    chopt=webdriver.ChromeOptions()
    appState = {
        "recentDestinations": [
            {
                "id": "Save as PDF",
                "origin": "local",
                "account":""
            }
        ],
        "selectedDestinationId": "Save as PDF",
        "version": 2,
        "isLandscapeEnabled": False, #印刷の向きを指定 tureで横向き、falseで縦向き。
        "pageSize": 'A4', #用紙タイプ(A3、A4、A5、Legal、 Letter、Tabloidなど)
        "isHeaderFooterEnabled": False, #ヘッダーとフッター
        "isCssBackgroundEnabled": True, #背景のグラフィック
        "isCollateEnabled": True #部単位で印刷
    }
    
    prefs = {'printing.print_preview_sticky_settings.appState':
             json.dumps(appState),
             "savefile.default_directory": output_dir
             } #appState --> pref
    chopt.add_experimental_option('prefs', prefs) #prefs --> chopt
    chopt.add_argument('--kiosk-printing') #印刷ダイアログが開くと、印刷ボタンを無条件に押す。
    return chopt

In [14]:
def main_WebToPDF(BlogURL, output_dir):
    #Web ページもしくはhtmlファイルをPDFにSeleniumを使って変換する
    chopt = PrintSetUp(output_dir)
    driver_path = "./chromedriver" #webdriverのパス
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chopt)   
#    driver = webdriver.Chrome(options=chopt)
#    driver = webdriver.Chrome(executable_path=driver_path, options=chopt)
    driver.implicitly_wait(1) # 秒 暗示的待機 
    driver.get(BlogURL) #ブログのURL 読み込み
    WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located)  # ページ上のすべての要素が読み込まれるまで待機（15秒でタイムアウト判定）
    time.sleep(2) #レンダリングを待つ
    driver.execute_script('return window.print()') #Print as PDF
    time.sleep(1) #ファイルのダウンロードのために10秒待機
    driver.quit() #Close Screen

In [15]:
from pathlib import Path
for html in glob.glob('../html/*.html'):
    main_WebToPDF('file://' + str(Path(html).resolve()), str(Path('../pdf').resolve()))
    



NameError: name 'ChromeDriverManager' is not defined